# Trabalho Final - Avaliação e Desempenho 2022.1
## Grupo:
- Matheus Fernandes Cabral - DRE: 116033208
- Rafael da Silva Fernandes - DRE: 117196229
- Stephanie Orazem Hoegemann Ramos - DRE: 113278168

## Objetivo 
O foco deste trabalho de simulação será a implementação de uma simulação orientada a eventos discretos e que permita a obtenção de intervalos de confiança para algumas métricas de uma fila M/M/1, usando as disciplinas de atendimento entre FCFS e LCFS.

## Descrição dos eventos

### Chegada

- Insere o freguês na fila de fregueses que chegaram ao sistema
- Programa um evento de chegada na fila de eventos, tempo dependendo da taxa de chegada Poisson
    - Se fila vazia:
        - programa um evento de entrada no serviço da fila de eventos, com o mesmo tempo dessa chegada
    - Do contrário:
        - insere o freguês na fila de espera

### Entrada em serviço

- Programa um evento de saída do serviço, com o tempo dado pela distribuição exponencial do serviço.
- Coleta estatísticas sobre o temo de espera na fila

### Saída de serviço

- Coleta estatísticas sobre o tempo de serviço
- Se a fila não estiver vazia, programa um evento de entrada em serviço na fila de eventos, com o mesmo tempo da saída de serviço

> ToDo:
- mostrar independência da semente inicial
- calcular sobreposição dos ICs pelas duas metodologias e comparar
- calcular valor central de cada intervalo
- relatório
- testes de correção
- estimativa da fase transiente

## Bibliotecas utilizadas

In [1]:
import numpy as np
import pandas as pd
import scipy.special as sc
from scipy.stats import chi2, t

## Variáveis globais

In [2]:
# Inicialização do identificador único dos fregueses
CUSTOMER_ID = 0

# Booleano para indicar se o servidor está ocupado (True) ou não (False)
BUSY_SERVER = False

# Número de rodadas, definido pelo enunciado do trabalho
N_ROUNDS = 3200

# Kmin >> 1, para que a média do tempo de espera por rodada possa ser assumido como distribuição Normal
KMIN = 100

# Lista com os valores de lambda a serem trabalhos
LAMBDAS = [0.2, 0.4, 0.6, 0.8, 0.9]

## Classes

### Freguês

In [3]:
class Customer:
    '''
    Atributos do freguês:
    - id: identificador único de cada freguês
    - arrival_time: tempo de chegada no sistema
    - entry_server_time: tempo de entrada em serviço
    - exit_server_time: tempo de saída do serviço
    - arrival_round: rodada em que o freguês chega
    '''

    def __init__(self, id, arrival_time, arrival_round):
        self.id = id
        self.arrival_time = arrival_time
        self.arrival_round = arrival_round
        self.entry_server_time = 0
        self.exit_server_time = 0

### Estatísticas

In [4]:
class Statistics:
    '''
    Somatório das estatísticas, sendo elas:
    - tempo de serviço
    - tempo de espera na fila
    - tempo total gasto no sistema
    '''

    def __init__(self):
        self.sample_index = 0
        self.sample_service_time = 0
        self.sample_queue_time = 0
        self.sample_system_time = 0
        self.mean_queue_wait = 0

    
    def statistics_accumulator(self, customer):
        self.sample_index += 1
        self.sample_service_time += customer.exit_server_time - customer.entry_server_time
        self.sample_queue_time += customer.entry_server_time - customer.arrival_time
        self.sample_system_time += customer.exit_server_time - customer.arrival_time

    
    def mean_calculator(self):
        '''
        Método responsável pelo cálculo da média do tempo de espera na fila
        '''
        self.mean_queue_wait = self.sample_queue_time/ self.sample_index

### Utilitários

In [5]:
class Utils:

    def append_event(event, event_list):
        '''
        Adiciona evento à lista de eventos
        '''
        
        for i in range(len(event_list)):
            
            # Caso o evento na posição i da lista de eventos tenha um tempo de início superior ao do evento passado como parâmetro
            # Nós inserimos o evento passado como parâmetro na posição i da lista, substituindo assim o original
            if(event_list[i].start_time >= event.start_time):
                event_list.insert(i, event)
                return
        
        event_list.append(event)


    def find_customer(customer_list, id):
        '''
        Encontra freguês na lista de fregueses com o id passado como parâmetro 
        '''

        for customer in range(len(customer_list)):
            if(customer_list[customer].id == id):
                return customer

        return None


    def generate_arrival_time(lambda_rate):
        '''
        Calcula tempo de chegada com base na taxa lambda passada como parâmetro
        '''

        u0 = np.random.uniform()
        t0 = np.log(u0)/ (-lambda_rate)

        return t0


    # A taxa de serviço é igual à 1 pois foi dada na descrição do trabalho
    def generate_service_time():
        '''
        Calcula tempo de serviço com taxa igual à 1
        '''

        u0 = np.random.uniform()
        x0 = np.log(u0)/ (-1)

        return x0


    def variance_queue_wait_confidence_interval(estimated_variance):
        '''
        Cálculo do Intervalo de Confiança (IC) utilizando a distribuição chi-quadrado
        '''

        # Cálculo dos limites inferior e superior
        superior_limit = ((N_ROUNDS - 1) * estimated_variance) / chi2.ppf(q = 0.025, df = N_ROUNDS - 1)
        inferior_limit = ((N_ROUNDS - 1) * estimated_variance) / chi2.ppf(q = 0.975, df = N_ROUNDS - 1)

        # Cálculo da precisão
        chi_sup = chi2.ppf(q = 0.025, df = N_ROUNDS-1)
        chi_inf =  chi2.ppf(q = 0.975, df = N_ROUNDS-1)
        precision = (chi_inf - chi_sup)/ (chi_sup + chi_inf)

        return inferior_limit, superior_limit , precision


    def mean_queue_wait_confidence_interval(standard_deviation, estimated_mean):
        '''
        Cálculo do IC usando a distribuição t-student
        '''

        # Cálculo dos limites inferior e superior
        superior_limit = estimated_mean + (t.ppf(q = 0.975, df = N_ROUNDS - 1) * (standard_deviation/ np.sqrt(N_ROUNDS))) 
        inferior_limit = estimated_mean - (t.ppf(q = 0.975, df = N_ROUNDS - 1) * (standard_deviation/ np.sqrt(N_ROUNDS)))
        
        # Cálculo da precisão
        precision = (superior_limit - inferior_limit)/ (superior_limit + inferior_limit)
        
        return inferior_limit, superior_limit, precision

### Evento

In [6]:
class Event:
    '''
    Atributos de cada evento:
    - event_type: tipo do evento
        - chegada no sistema (CH)
        - entrada em serviço (ES)
        - saída de serviço (SS)
    - start_time: tempo de início do evento
    - customer_index: índice do freguês a qual o evento se refere
    '''

    def __init__(self, event_type, start_time, customer_index):
        self.event_type = event_type
        self.start_time = start_time
        self.customer_index = customer_index

    
    def queue_arrival(self, customer_list, event_list, wait_queue, current_round, lambda_rate):
        '''
        Fila de chegada
        '''

        # Permite alterar o valor da variável global CUSTOMER_ID dentro da função
        global CUSTOMER_ID

        arrival_time = self.start_time + Utils.generate_arrival_time(lambda_rate)

        # Incrementa índice do freguês, o instancia e adiciona na lista de fregueses
        CUSTOMER_ID += 1
        customer_list.append(Customer(CUSTOMER_ID, arrival_time, current_round))

        # Adiciona chegada no sistema à fila de eventos
        Utils.append_event(Event('CH', arrival_time, CUSTOMER_ID), event_list)

        # Caso não haja ninguém na fila de espera e o servidor não esteja vazio
        # Adiciona entrada em serviço à fila de eventos
        if(len(wait_queue) == 0 and not BUSY_SERVER):
            Utils.append_event(Event('ES', self.start_time, self.customer_index), event_list)
        
        # Caso contrário, adiciona freguês à lista de espera
        else:
            wait_queue.append(self.customer_index)


    def service_entry(self, customer_list, event_list, wait_queue, FCFS = True):
        '''
        Calcula a entrada do serviço
        '''

        global BUSY_SERVER

        # Caso a fila não esteja vazia, pega-se o freguês com base na disciplina
        if(len(wait_queue) > 0):
            
            # Com base em FCFS, pega-se o primeiro freguês
            if(FCFS):
                wait_queue.pop(0)
            # Com base em LCFS, pega-se o último freguês
            else:
                wait_queue.pop(-1)
        
        # Calcula tempo de serviço total
        service_time = self.start_time + Utils.generate_service_time()
        
        # Adiciona saída de serviço à fila de eventos
        Utils.append_event(Event('SS', service_time, self.customer_index), event_list)

        # Sinaliza que o servidor está ocupado
        BUSY_SERVER = True

        # Relaciona o freguês com o seu tempo de serviço
        customer_list[Utils.find_customer(customer_list, self.customer_index)].entry_server_time = self.start_time


    def service_exit(self, customer_list, event_list, wait_queue, statistics, current_round, FCFS = True):
        '''
        Calcula a saída do serviço
        '''

        global BUSY_SERVER

        # Caso a fila não esteja vazia, adiciona entrada em serviço à fila de eventos
        if(len(wait_queue) > 0):
            # Com base em FCFS
            if(FCFS):
                Utils.append_event(Event('ES', self.start_time, wait_queue[0]), event_list)
            # Com base em LCFS
            else:
                Utils.append_event(Event('ES', self.start_time, wait_queue[-1]), event_list)

        # Sinaliza que o servidor não está ocupado
        BUSY_SERVER = False

        # Calculam as estatísticas do freguês e o remove da lista de fregueses 
        aux_customer_id = Utils.find_customer(customer_list, self.customer_index)
        customer_list[aux_customer_id].exit_server_time = self.start_time
        statistics[current_round].statistics_accumulator(customer_list[aux_customer_id])
        customer_list.pop(aux_customer_id)

## Execução do simulador

### Fluxo principal

In [7]:
from statistics import variance


def main(lambda_rate, FCFS = True):

    # Inicialização das variáveis média e desvios quadrados estimados
    estimated_mean, square_deviations = 0, 0

    # Listas:
    # Lista dos eventos | Lista dos fregueses | Fila de espera | Estatísticas
    event_list, customer_list, wait_queue, statistics = [], [], [], []
    
    # Definindo a primeira chegada no sistema
    FIRST_CUSTOMER = Customer(0, 0, 0)
    customer_list.append(FIRST_CUSTOMER)

    FIRST_ARRIVAL = Event('CH', 0, 0)
    event_list.append(FIRST_ARRIVAL)

    for current_round in range(N_ROUNDS):

        statistics.append(Statistics())

        while(statistics[current_round].sample_index < KMIN):
            
            # Pega primeiro evento da lista de eventos
            current_event = event_list.pop(0)

            # Caso o tipo do evento seja chegada ao sistema
            if(current_event.event_type == 'CH'):
                current_event.queue_arrival(customer_list, event_list, wait_queue, current_round, lambda_rate)

            # Caso o tipo do evento seja entrada em serviço
            elif(current_event.event_type == 'ES'):
                current_event.service_entry(customer_list, event_list, wait_queue, FCFS)

            # Caso o tipo do evento seja saída do serviço
            elif(current_event.event_type == 'SS'):
                current_event.service_exit(customer_list, event_list, wait_queue, statistics, current_round, FCFS)

        # Cálcula média das estatísticas para a rodada atual
        statistics[current_round].mean_calculator()


    # Dicionário para os resultados que devem ser retornados
    results_dict = {}

    # Cálculo da média estimada
    for statistic in statistics:
        estimated_mean += statistic.mean_queue_wait

    # Cálculo da média real
    real_mean = estimated_mean/ N_ROUNDS
    results_dict["Real mean"] = real_mean

    # Cálculo dos desvios quadrados
    for statistic in statistics:
        square_deviations += (statistic.mean_queue_wait - real_mean) ** 2

    # Cálculo da variância
    variance = square_deviations/ (N_ROUNDS - 1)
    results_dict["Variance"] = variance

    # Cálculo dos limites inferior, superior e precisão da distribuição chi-quadrado
    infe_limit, sup_limit, chi_precision = Utils.variance_queue_wait_confidence_interval(variance)
    results_dict["Chi Inferior limit"] = infe_limit
    results_dict["Chi Superior limit"] = sup_limit
    results_dict["Chi Precision"] = chi_precision

    # Cálculo dos limites inferior, superior e precisão da distribuição t-student
    mean_infe_limit, mean_sup_limit, t_precision = Utils.mean_queue_wait_confidence_interval(np.sqrt(variance), real_mean)
    results_dict["T Student Inferior limit"] = mean_infe_limit
    results_dict["T Student Superior limit"] = mean_sup_limit
    results_dict["T Student Precision"] = t_precision

    # Transformando o dicionário em um DataFrame
    results_dataframe = pd.DataFrame.from_dict(results_dict, orient = "index")

    return results_dataframe

### Resultados gerais

#### FCFS

In [8]:
%%time
results_fcfs = pd.DataFrame()

while(KMIN <= 1000):
    for lambda_rate in LAMBDAS:
        results_fcfs["Lambda = " + str(lambda_rate)] = main(lambda_rate)

    results_fcfs.rename_axis("K = " + str(KMIN), inplace = True)
    display(results_fcfs)

    KMIN += 100

,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 100,,,,,
Real mean,0.248909,0.662863,1.517514,3.880847,8.735749
Variance,0.013436,0.101896,0.857239,9.635432,69.459800
Chi Inferior limit,0.012801,0.097081,0.816732,9.180132,66.177641
Chi Superior limit,0.014119,0.107080,0.900851,10.125633,72.993560
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.244891,0.651798,1.485422,3.773256,8.446878
T Student Superior limit,0.252926,0.673927,1.549605,3.988437,9.024620
T Student Precision,0.016141,0.016691,0.021147,0.027723,0.033068


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 200,,,,,
Real mean,0.248132,0.667252,1.489473,3.941137,8.968996
Variance,0.006669,0.052720,0.383896,7.214680,60.979817
Chi Inferior limit,0.006354,0.050229,0.365756,6.873767,58.098359
Chi Superior limit,0.007008,0.055402,0.403426,7.581726,64.082159
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.245302,0.659293,1.467998,3.848038,8.698332
T Student Superior limit,0.250963,0.675210,1.510949,4.034236,9.239660
T Student Precision,0.011407,0.011927,0.014418,0.023622,0.030178


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 300,,,,,
Real mean,0.250605,0.665286,1.492174,4.008486,9.471879
Variance,0.004588,0.035967,0.279125,5.164177,70.753934
Chi Inferior limit,0.004372,0.034267,0.265935,4.920156,67.410624
Chi Superior limit,0.004822,0.037797,0.293325,5.426904,74.353533
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.248257,0.658712,1.473862,3.929720,9.180329
T Student Superior limit,0.252953,0.671859,1.510486,4.087252,9.763428
T Student Precision,0.009369,0.009881,0.012272,0.019650,0.030781


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 400,,,,,
Real mean,0.249691,0.664614,1.510037,4.065665,9.110248
Variance,0.003304,0.026375,0.225327,5.096806,50.053044
Chi Inferior limit,0.003148,0.025129,0.214680,4.855968,47.687905
Chi Superior limit,0.003472,0.027717,0.236790,5.356105,52.599487
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.247698,0.658985,1.493584,3.987414,8.865030
T Student Superior limit,0.251683,0.670243,1.526490,4.143915,9.355466
T Student Precision,0.007979,0.008470,0.010896,0.019247,0.026917


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 500,,,,,
Real mean,0.249498,0.662504,1.503709,3.979643,9.368817
Variance,0.002786,0.020434,0.171029,3.682308,51.861457
Chi Inferior limit,0.002654,0.019469,0.162947,3.508309,49.410866
Chi Superior limit,0.002928,0.021474,0.179730,3.869645,54.499903
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.247669,0.657549,1.489375,3.913132,9.119208
T Student Superior limit,0.251328,0.667459,1.518043,4.046155,9.618426
T Student Precision,0.007332,0.007479,0.009533,0.016713,0.026643


...

,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 700,,,,,
Real mean,0.250026,0.668467,1.501866,4.032901,8.908132
Variance,0.001979,0.016406,0.124149,2.824341,32.449079
Chi Inferior limit,0.001886,0.015631,0.118283,2.690883,30.915774
Chi Superior limit,0.002080,0.017241,0.130466,2.968029,34.099922
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.248484,0.664028,1.489654,3.974651,8.710690
T Student Superior limit,0.251568,0.672907,1.514079,4.091151,9.105574
T Student Precision,0.006168,0.006641,0.008132,0.014444,0.022164


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 800,,,,,
Real mean,0.249462,0.666896,1.504456,4.004017,9.159259
Variance,0.001779,0.013240,0.107196,2.426400,40.294059
Chi Inferior limit,0.001695,0.012614,0.102131,2.311746,38.390058
Chi Superior limit,0.001869,0.013913,0.112650,2.549843,42.344015
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.248000,0.662908,1.493108,3.950026,8.939241
T Student Superior limit,0.250924,0.670884,1.515805,4.058008,9.379277
T Student Precision,0.005860,0.005980,0.007543,0.013484,0.024021


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 900,,,,,
Real mean,0.250772,0.666726,1.502233,3.976376,8.963302
Variance,0.001610,0.011266,0.096168,2.115018,27.907489
Chi Inferior limit,0.001534,0.010733,0.091623,2.015077,26.588786
Chi Superior limit,0.001692,0.011839,0.101060,2.222619,29.327279
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.249381,0.663047,1.491485,3.925969,8.780198
T Student Superior limit,0.252163,0.670404,1.512982,4.026784,9.146406
T Student Precision,0.005547,0.005518,0.007155,0.012677,0.020428


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 1000,,,,,
Real mean,0.249641,0.667821,1.494062,4.005020,9.000063
Variance,0.001440,0.010417,0.077892,1.998286,28.514333
Chi Inferior limit,0.001372,0.009925,0.074212,1.903862,27.166955
Chi Superior limit,0.001513,0.010947,0.081855,2.099949,29.964997
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.248325,0.664283,1.484389,3.956023,8.814979
T Student Superior limit,0.250956,0.671359,1.503736,4.054017,9.185147
T Student Precision,0.005269,0.005297,0.006475,0.012234,0.020565


CPU times: total: 18min 48s
Wall time: 19min 12s


#### LCFS

In [9]:
%%time
KMIN = 100
results_lcfs = pd.DataFrame()

while(KMIN <= 1000):
    for lambda_rate in LAMBDAS:
        results_lcfs["Lambda = " + str(lambda_rate)] = main(lambda_rate, FCFS = False)
    
    results_lcfs.rename_axis("K = " + str(KMIN), inplace = True)
    display(results_lcfs)
    
    KMIN += 100

,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 100,,,,,
Real mean,0.248103,0.674473,1.490780,3.965975,8.759270
Variance,0.013523,0.097483,0.817123,14.361183,136.526229
Chi Inferior limit,0.012884,0.092876,0.778511,13.682579,130.075004
Chi Superior limit,0.014211,0.102442,0.858694,15.091807,143.471987
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.244072,0.663651,1.459449,3.834624,8.354278
T Student Superior limit,0.252133,0.685295,1.522112,4.097326,9.164261
T Student Precision,0.016246,0.016045,0.021017,0.033119,0.046236


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 200,,,,,
Real mean,0.248248,0.663858,1.503113,3.960909,8.804698
Variance,0.006853,0.049493,0.447445,7.638231,99.247994
Chi Inferior limit,0.006529,0.047154,0.426302,7.277305,94.558263
Chi Superior limit,0.007202,0.052011,0.470209,8.026826,104.297225
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.245379,0.656147,1.479928,3.865116,8.459396
T Student Superior limit,0.251117,0.671569,1.526298,4.056702,9.149999
T Student Precision,0.011558,0.011615,0.015425,0.024185,0.039218


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 300,,,,,
Real mean,0.250300,0.662847,1.513736,4.016556,8.588020
Variance,0.004668,0.035304,0.308890,6.124133,61.757576
Chi Inferior limit,0.004447,0.033636,0.294295,5.834751,58.839367
Chi Superior limit,0.004905,0.037100,0.324605,6.435697,64.899486
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.247932,0.656335,1.494473,3.930781,8.315636
T Student Superior limit,0.252669,0.669360,1.533000,4.102330,8.860405
T Student Precision,0.009461,0.009825,0.012726,0.021355,0.031717


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 400,,,,,
Real mean,0.248578,0.669829,1.497352,4.033539,8.961535
Variance,0.003381,0.026203,0.212183,4.708429,70.700567
Chi Inferior limit,0.003222,0.024965,0.202157,4.485943,67.359778
Chi Superior limit,0.003553,0.027536,0.222978,4.947970,74.297451
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.246563,0.664219,1.481386,3.958329,8.670095
T Student Superior limit,0.250594,0.675440,1.513318,4.108749,9.252974
T Student Precision,0.008108,0.008376,0.010663,0.018646,0.032521


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 500,,,,,
Real mean,0.249383,0.667333,1.498305,3.971540,9.050670
Variance,0.002889,0.020364,0.184334,3.485011,57.392004
Chi Inferior limit,0.002753,0.019402,0.175624,3.320335,54.680080
Chi Superior limit,0.003036,0.021400,0.193712,3.662310,60.311816
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.247520,0.662386,1.483424,3.906835,8.788089
T Student Superior limit,0.251246,0.672279,1.513186,4.036245,9.313251
T Student Precision,0.007471,0.007412,0.009932,0.016292,0.029012


...

,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 700,,,,,
Real mean,0.250002,0.669186,1.494704,4.017061,9.101245
Variance,0.001937,0.014590,0.122714,2.693838,47.136617
Chi Inferior limit,0.001846,0.013901,0.116915,2.566547,44.909287
Chi Superior limit,0.002036,0.015333,0.128957,2.830886,49.534688
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.248476,0.664999,1.482562,3.960173,8.863278
T Student Superior limit,0.251528,0.673372,1.506846,4.073950,9.339212
T Student Precision,0.006102,0.006256,0.008123,0.014162,0.026147


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 800,,,,,
Real mean,0.249227,0.667567,1.500149,3.967264,8.807988
Variance,0.001742,0.012781,0.115459,2.220514,33.370832
Chi Inferior limit,0.001660,0.012177,0.110003,2.115588,31.793972
Chi Superior limit,0.001830,0.013431,0.121333,2.333482,35.068569
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.247781,0.663648,1.488372,3.915615,8.607762
T Student Superior limit,0.250674,0.671485,1.511926,4.018914,9.008214
T Student Precision,0.005804,0.005870,0.007851,0.013019,0.022732


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 900,,,,,
Real mean,0.249737,0.667287,1.498784,4.042976,8.956161
Variance,0.001462,0.011704,0.104917,2.183754,34.364988
Chi Inferior limit,0.001393,0.011151,0.099959,2.080566,32.741151
Chi Superior limit,0.001536,0.012299,0.110254,2.294852,36.113303
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.248412,0.663537,1.487557,3.991756,8.752974
T Student Superior limit,0.251062,0.671036,1.510011,4.094196,9.159348
T Student Precision,0.005306,0.005619,0.007491,0.012669,0.022687


,Lambda = 0.2,Lambda = 0.4,Lambda = 0.6,Lambda = 0.8,Lambda = 0.9
K = 1000,,,,,
Real mean,0.250822,0.670476,1.504227,3.962678,8.856871
Variance,0.001425,0.011228,0.086519,1.741599,28.935160
Chi Inferior limit,0.001358,0.010698,0.082431,1.659304,27.567898
Chi Superior limit,0.001498,0.011799,0.090921,1.830202,30.407234
Chi Precision,0.048975,0.048975,0.048975,0.048975,0.048975
T Student Inferior limit,0.249513,0.666803,1.494032,3.916937,8.670426
T Student Superior limit,0.252130,0.674149,1.514422,4.008420,9.043316
T Student Precision,0.005217,0.005478,0.006778,0.011543,0.021051


CPU times: total: 19min 13s
Wall time: 19min 40s


## Questões

### a) Tempo médio de espera em fila

#### FCFS

#### LCFS

### b) Variância do tempo de espera em fila

#### FCFS

#### LCFS

### c) Número médio na fila de espera

#### FCFS

#### LCFS

### d) Variância do número de pessoas na fila de espera

#### FCFS

#### LCFS